In [1]:
import gzip
import itertools
import json
import math
import pickle
from typing import Iterator, List

import pandas as pd

# Table of Contents

- [Step C.1](#Step-C.1)
- [Step C.2](#Step-C.2)

## Step C.1

In [2]:
# disclaimer: may not work as is in Windows OS
# download the metadata for this dataset
# dataset source: https://nijianmo.github.io/amazon/index.html

!wget --backups=1 http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_Digital_Music.json.gz -P data/

--2022-03-20 12:27:54--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_Digital_Music.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12367273 (12M) [application/octet-stream]
Saving to: ‘data/meta_Digital_Music.json.gz’

meta_Digital_Music. 100%[===================>]  11.79M  35.5KB/s    in 4m 33s  

2022-03-20 12:32:27 (44.2 KB/s) - ‘data/meta_Digital_Music.json.gz’ saved [12367273/12367273]



In [2]:
def inspect_df(df: pd.DataFrame, n: int = 5) -> pd.DataFrame:
    """Helper method to easily inspect DataFrames."""

    print(f"shape: {df.shape}")

    return df.head(n)

In [3]:
def parse(filepath: str) -> Iterator[dict]:
    file_obj = gzip.open(filepath, "rb")
    for line in file_obj:
        yield json.loads(line)

In [4]:
def file_to_dataframe(filepath: str) -> pd.DataFrame:
    i = 0
    df = {}
    for d in parse(filepath):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient="index")

In [5]:
metadata = file_to_dataframe("data/meta_Digital_Music.json.gz")

inspect_df(metadata)

shape: (74347, 19)


,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes,details
0,[],,[],,Master Collection Volume One,"[B000002UEN, B000008LD5, B01J804JKE, 747403435...",,John Michael Talbot,[],"58,291 in CDs & Vinyl (","[B000002UEN, B000008LD5, 7474034352, B000008LD...","<img src=""https://images-na.ssl-images-amazon....",,,$18.99,0001377647,[],[],NaN
1,[],,[],,Hymns Collection: Hymns 1 &amp; 2,"[5558154950, B00014K5V4]",,Second Chapter of Acts,[],"93,164 in CDs & Vinyl (","[B000008KJ3, B000008KJ0, 5558154950, B000UN8KZ...","<img src=""https://images-na.ssl-images-amazon....",,,,0001529145,[],[],NaN
2,[],,[],,Early Works - Don Francisco,"[B00004RC05, B003H8F4NA, B003ZFVHPO, B003JMP1Z...",,Don Francisco,[],"875,825 in CDs & Vinyl (","[B003H8F4NA, B003ZFVHPO, B003JMP1ZK, B00004RC0...","<img src=""https://images-na.ssl-images-amazon....",,,,0001527134,[],[],NaN
3,[],,[],,So You Wanna Go Back to Egypt,"[B0000275QQ, 0001393774, 0001388312, B0016CP2G...",,Keith Green,[],"203,263 in CDs & Vinyl (","[B00000I7JO, B0016CP2GS, 0001393774, B0000275Q...","<img src=""https://images-na.ssl-images-amazon....",,,$13.01,0001388703,[],[],NaN
4,[],,[1. Losing Game 2. I Can't Wait 3. Didn't He S...,,Early Works - Dallas Holm,"[B0002N4JP2, 0760131694, B00002EQ79, B00150K8J...",,Dallas Holm,[],"399,269 in CDs & Vinyl (","[B0002N4JP2, 0760131694, B00150K8JC, B003MTXNV...","<img src=""https://images-na.ssl-images-amazon....",,,,0001526146,[],[],NaN


In [6]:
naive_df = pickle.load(open("data/naive-RS.pkl", "rb"))
content_based_df = pickle.load(open("data/content-based-RS.pkl", "rb"))
collab_filtering_df = pickle.load(open("data/collab-filtering-RS.pkl", "rb"))

In [7]:
naive_df.rename(
    columns={"reviewerID": "user", "asin": "top_recommendations"}, inplace=True
)
content_based_df.rename(
    columns={"reviewerID": "user", "asin": "top_recommendations"}, inplace=True
)

In [8]:
inspect_df(content_based_df)

shape: (167, 2)


,user,top_recommendations
0,A1352I3HWDQCZH,"[B01AB9Q2RE, B00000I7LP, B000X1HU68, 971472118..."
1,A15GBKY0IPZJI3,"[B01AB9Q2RE, B00000I7LP, B000X1HU68, 971472118..."
2,A15JTJXQXO22JJ,"[B01AB9Q2RE, B00000I7LP, B000X1HU68, B0016DCUI..."
3,A15OXG4V7IK2D9,"[B01AB9Q2RE, B00000I7LP, B000X1HU68, 971472118..."
4,A17B6IPLJ964N0,"[B01AB9Q2RE, B00000I7LP, B000X1HU68, 971472118..."


In [9]:
inspect_df(collab_filtering_df)

shape: (16566, 2)


,user,top_recommendations
0,A2TYZ821XXK2YZ,"[7799420340, 9714721180, B00004TTUZ, B000EO8VM..."
1,A3OFSREZADFUDY,"[7799420340, B0002HA8IA, B000BD5OPM, B000BD8WU..."
2,A2VAMODP8M77NG,"[7799420340, B000BD8WUQ, B000QO6B1C, B000S43QJ..."
3,AAKSLZ9IDTEH0,"[B007LAZ7L4, B001NYEHT2, B0012ECGIC, B001NJUGV..."
4,A3OH43OZJLKI09,"[B001BZF788, B00137MGM4, B002XMJ7IO, B0011W8V2..."


In [10]:
inspect_df(naive_df)

shape: (167, 2)


,user,top_recommendations
0,A1352I3HWDQCZH,[B000W2S5X0]
1,A15GBKY0IPZJI3,[B009XJVKIU]
2,A15JTJXQXO22JJ,[B00073QG7G]
3,A15OXG4V7IK2D9,[B0017TDP3C]
4,A17B6IPLJ964N0,[B0055HVX7W]


In [11]:
all_ratings = pd.DataFrame()
all_ratings["user"] = naive_df["user"]
all_ratings["top_5_content"] = all_ratings["user"].map(
    lambda x: content_based_df[content_based_df["user"] == x][
        "top_recommendations"
    ].values[0][0:5]
)
all_ratings["top_5_collab"] = all_ratings["user"].map(
    lambda x: collab_filtering_df[collab_filtering_df["user"] == x][
        "top_recommendations"
    ].values[0][0:5]
)
all_ratings["all_content"] = all_ratings["user"].map(
    lambda x: content_based_df[content_based_df["user"] == x][
        "top_recommendations"
    ].values[0]
)
all_ratings["all_collab"] = all_ratings["user"].map(
    lambda x: collab_filtering_df[collab_filtering_df["user"] == x][
        "top_recommendations"
    ].values[0]
)
all_ratings["naive"] = all_ratings["user"].map(
    lambda x: naive_df[naive_df["user"] == x]["top_recommendations"].values[0]
)

In [12]:
inspect_df(all_ratings)

shape: (167, 6)


,user,top_5_content,top_5_collab,all_content,all_collab,naive
0,A1352I3HWDQCZH,"[B01AB9Q2RE, B00000I7LP, B000X1HU68, 971472118...","[B000T31G8S, B000V66LII, B000VBH2R2, B000VIP5R...","[B01AB9Q2RE, B00000I7LP, B000X1HU68, 971472118...","[B000T31G8S, B000V66LII, B000VBH2R2, B000VIP5R...",[B000W2S5X0]
1,A15GBKY0IPZJI3,"[B01AB9Q2RE, B00000I7LP, B000X1HU68, 971472118...","[9714721180, B000EO8VMQ, B000QMG34E, B000QVL4N...","[B01AB9Q2RE, B00000I7LP, B000X1HU68, 971472118...","[9714721180, B000EO8VMQ, B000QMG34E, B000QVL4N...",[B009XJVKIU]
2,A15JTJXQXO22JJ,"[B01AB9Q2RE, B00000I7LP, B000X1HU68, B0016DCUI...","[B00JGEU434, 9714721180, B001EE4PUM, B0059CQ3Q...","[B01AB9Q2RE, B00000I7LP, B000X1HU68, B0016DCUI...","[B00JGEU434, 9714721180, B001EE4PUM, B0059CQ3Q...",[B00073QG7G]
3,A15OXG4V7IK2D9,"[B01AB9Q2RE, B00000I7LP, B000X1HU68, 971472118...","[B000V61GBA, B00CMBUOJW, B00K5RU72O, B005OC6A9...","[B01AB9Q2RE, B00000I7LP, B000X1HU68, 971472118...","[B000V61GBA, B00CMBUOJW, B00K5RU72O, B005OC6A9...",[B0017TDP3C]
4,A17B6IPLJ964N0,"[B01AB9Q2RE, B00000I7LP, B000X1HU68, 971472118...","[9714721180, B000S55UPC, B000TDUQ48, B000V66HB...","[B01AB9Q2RE, B00000I7LP, B000X1HU68, 971472118...","[9714721180, B000S55UPC, B000TDUQ48, B000V66HB...",[B0055HVX7W]


In [13]:
all_ratings_titles = pd.DataFrame()
all_ratings_titles["user"] = naive_df["user"]
all_ratings_titles["top_5_content_title"] = all_ratings["top_5_content"].map(
    lambda x: list(
        itertools.chain(*[metadata[metadata["asin"] == i]["title"] for i in x])
    )
)
all_ratings_titles["top_5_collab_title"] = all_ratings["top_5_collab"].map(
    lambda x: list(
        itertools.chain(*[metadata[metadata["asin"] == i]["title"] for i in x])
    )
)
all_ratings_titles["naive_title"] = all_ratings["naive"].map(
    lambda x: list(
        itertools.chain(*[metadata[metadata["asin"] == i]["title"] for i in x])
    )
)

In [14]:
all_ratings_titles

,user,top_5_content_title,top_5_collab_title,naive_title
0,A1352I3HWDQCZH,"[Say It To Me: EP/CD-single - UK Release, Slim...",[],[Joe Satriani - Surfing With The Alien - Lp Vi...
1,A15GBKY0IPZJI3,"[Say It To Me: EP/CD-single - UK Release, Slim...","[<span class=""a-size-medium a-color-secondary ...",[Still Believe]
2,A15JTJXQXO22JJ,"[Say It To Me: EP/CD-single - UK Release, Slim...","[<span class=""a-size-medium a-color-secondary ...",[Cross Purposes Live]
3,A15OXG4V7IK2D9,"[Say It To Me: EP/CD-single - UK Release, Slim...",[Dave's Picks Volume 6],[Wake Up the World]
4,A17B6IPLJ964N0,"[Say It To Me: EP/CD-single - UK Release, Slim...","[<span class=""a-size-medium a-color-secondary ...","[<span class=""a-size-medium a-color-secondary ..."
...,...,...,...,...
162,AWG2O9C42XW5G,"[Say It To Me: EP/CD-single - UK Release, This...","[Slayer - Greatest Hits 2 CD Set, He Touched M...",[Polka Party!]
163,AWJ9J0JAHN6PQ,"[Say It To Me: EP/CD-single - UK Release, Slim...",[],[Shades Of Time]
164,AWMORCDEUIBFO,"[Long Lonesome Highway, Road Trips, Vol. 1, No...",[],"[Long Road, Long Road]"
165,AXOO7BPM22BDO,"[Say It To Me: EP/CD-single - UK Release, Slim...",[],"[Nothing Has Changed The Best Of David Bowie, ..."


In [15]:
def get_ranking_positions_for_content_based(id_: str):
    top_5 = all_ratings[all_ratings["user"] == id_]["top_5_content"].values[0]
    all_rec = all_ratings[all_ratings["user"] == id_]["all_collab"].values[0]
    reccomendations_positions = {}
    positions = []

    for i in range(len(top_5)):
        top_5_index = i
        try:
            all_rec_index = all_rec.index(top_5[i])
        except ValueError:
            all_rec_index = -1
        if all_rec_index > 0:
            pos = round(abs(top_5_index - all_rec_index) / 2)
        else:
            pos = all_rec_index
        positions.append(pos)

    reccomendations_positions = dict(zip(top_5, positions))
    return reccomendations_positions

In [16]:
def get_ranking_positions_for_collab(id_: str):
    top_5 = all_ratings[all_ratings["user"] == id_]["top_5_collab"].values[0]
    all_rec = all_ratings[all_ratings["user"] == id_]["all_content"].values[0]
    reccomendations_positions = {}
    positions = []

    for i in range(len(top_5)):
        top_5_index = i
        try:
            all_rec_index = all_rec.index(top_5[i])
        except ValueError:
            all_rec_index = -1
        if all_rec_index > 0:
            pos = round(abs(top_5_index - all_rec_index) / 2)
        else:
            pos = all_rec_index
        positions.append(pos)

    reccomendations_positions = dict(zip(top_5, positions))
    return reccomendations_positions

In [17]:
def get_final_ranking_for_user(id_: str):
    collab_ranking = get_ranking_positions_for_collab(id_)
    content_ranking = get_ranking_positions_for_content_based(id_)
    all_rankins_dict = {**content_ranking, **collab_ranking}
    final_ranking_list = list(
        dict(sorted(all_rankins_dict.items(), key=lambda x: x[1], reverse=True)).keys()
    )

    return final_ranking_list

In [18]:
all_ratings["ranking_final"] = all_ratings["user"].map(
    lambda x: get_final_ranking_for_user(x)
)

inspect_df(all_ratings)

shape: (167, 7)


,user,top_5_content,top_5_collab,all_content,all_collab,naive,ranking_final
0,A1352I3HWDQCZH,"[B01AB9Q2RE, B00000I7LP, B000X1HU68, 971472118...","[B000T31G8S, B000V66LII, B000VBH2R2, B000VIP5R...","[B01AB9Q2RE, B00000I7LP, B000X1HU68, 971472118...","[B000T31G8S, B000V66LII, B000VBH2R2, B000VIP5R...",[B000W2S5X0],"[B01AB9Q2RE, B00000I7LP, B000X1HU68, 971472118..."
1,A15GBKY0IPZJI3,"[B01AB9Q2RE, B00000I7LP, B000X1HU68, 971472118...","[9714721180, B000EO8VMQ, B000QMG34E, B000QVL4N...","[B01AB9Q2RE, B00000I7LP, B000X1HU68, 971472118...","[9714721180, B000EO8VMQ, B000QMG34E, B000QVL4N...",[B009XJVKIU],"[B000EO8VMQ, 9714721180, B01AB9Q2RE, B00000I7L..."
2,A15JTJXQXO22JJ,"[B01AB9Q2RE, B00000I7LP, B000X1HU68, B0016DCUI...","[B00JGEU434, 9714721180, B001EE4PUM, B0059CQ3Q...","[B01AB9Q2RE, B00000I7LP, B000X1HU68, B0016DCUI...","[B00JGEU434, 9714721180, B001EE4PUM, B0059CQ3Q...",[B00073QG7G],"[B01AB9Q2RE, B00000I7LP, B000X1HU68, B0016DCUI..."
3,A15OXG4V7IK2D9,"[B01AB9Q2RE, B00000I7LP, B000X1HU68, 971472118...","[B000V61GBA, B00CMBUOJW, B00K5RU72O, B005OC6A9...","[B01AB9Q2RE, B00000I7LP, B000X1HU68, 971472118...","[B000V61GBA, B00CMBUOJW, B00K5RU72O, B005OC6A9...",[B0017TDP3C],"[B00CMBUOJW, B01AB9Q2RE, B00000I7LP, B000X1HU6..."
4,A17B6IPLJ964N0,"[B01AB9Q2RE, B00000I7LP, B000X1HU68, 971472118...","[9714721180, B000S55UPC, B000TDUQ48, B000V66HB...","[B01AB9Q2RE, B00000I7LP, B000X1HU68, 971472118...","[9714721180, B000S55UPC, B000TDUQ48, B000V66HB...",[B0055HVX7W],"[9714721180, B01AB9Q2RE, B00000I7LP, B000X1HU6..."


In [19]:
all_ratings_titles["ranking_final_title"] = all_ratings["ranking_final"].map(
    lambda x: list(
        itertools.chain(*[metadata[metadata["asin"] == i]["title"] for i in x])
    )
)

inspect_df(all_ratings_titles)

shape: (167, 5)


,user,top_5_content_title,top_5_collab_title,naive_title,ranking_final_title
0,A1352I3HWDQCZH,"[Say It To Me: EP/CD-single - UK Release, Slim...",[],[Joe Satriani - Surfing With The Alien - Lp Vi...,"[Say It To Me: EP/CD-single - UK Release, Slim..."
1,A15GBKY0IPZJI3,"[Say It To Me: EP/CD-single - UK Release, Slim...","[<span class=""a-size-medium a-color-secondary ...",[Still Believe],"[A Salute to America, <span class=""a-size-medi..."
2,A15JTJXQXO22JJ,"[Say It To Me: EP/CD-single - UK Release, Slim...","[<span class=""a-size-medium a-color-secondary ...",[Cross Purposes Live],"[Say It To Me: EP/CD-single - UK Release, Slim..."
3,A15OXG4V7IK2D9,"[Say It To Me: EP/CD-single - UK Release, Slim...",[Dave's Picks Volume 6],[Wake Up the World],"[Dave's Picks Volume 6, Say It To Me: EP/CD-si..."
4,A17B6IPLJ964N0,"[Say It To Me: EP/CD-single - UK Release, Slim...","[<span class=""a-size-medium a-color-secondary ...","[<span class=""a-size-medium a-color-secondary ...","[<span class=""a-size-medium a-color-secondary ..."


## Step C.2

In [20]:
def compare_recommendations_for_user(id_: str):

    hybrid_user = pd.DataFrame()
    hybrid_user["asin"] = all_ratings.loc[all_ratings["user"] == id_][
        "ranking_final"
    ].values[0]
    hybrid_user["title"] = pd.DataFrame(
        all_ratings_titles.loc[all_ratings_titles["user"] == id_][
            "ranking_final_title"
        ].values[0]
    )

    content_user = pd.DataFrame()
    content_user["asin"] = all_ratings.loc[all_ratings["user"] == id_][
        "top_5_content"
    ].values[0]
    content_user["title"] = pd.DataFrame(
        all_ratings_titles.loc[all_ratings_titles["user"] == id_][
            "top_5_content_title"
        ].values[0]
    )

    naive_user = pd.DataFrame()
    naive_user["asin"] = all_ratings.loc[all_ratings["user"] == id_]["naive"].values[0]
    naive_user["title"] = pd.DataFrame(
        all_ratings_titles.loc[all_ratings_titles["user"] == id_]["naive_title"].values[
            0
        ]
    )

    print(f"For user {id_} the recommendations are:")
    print("\n")
    print("HYBRID\n")
    print(hybrid_user)
    print("=" * 100)
    print("CONTENT-BASED\n")
    print(content_user)
    print("=" * 100)
    print("NAIVE\n")
    print(naive_user)

In [21]:
compare_recommendations_for_user("A15OXG4V7IK2D9")

For user A15OXG4V7IK2D9 the recommendations are:


HYBRID

         asin                                              title
0  B00CMBUOJW                              Dave's Picks Volume 6
1  B01AB9Q2RE            Say It To Me: EP/CD-single - UK Release
2  B00000I7LP                                         Slim Shady
3  B000X1HU68                                   This Is Thirteen
4  9714721180  <span class="a-size-medium a-color-secondary a...
5  B0016DCUIA                    Invasion Of The Booty Snatchers
6  B000V61GBA                                                NaN
7  B00K5RU72O                                                NaN
8  B005OC6A9Y                                                NaN
9  B00123KDR4                                                NaN
CONTENT-BASED

         asin                                              title
0  B01AB9Q2RE            Say It To Me: EP/CD-single - UK Release
1  B00000I7LP                                         Slim Shady
2  B000X1HU68   

In [22]:
compare_recommendations_for_user("A1352I3HWDQCZH")

For user A1352I3HWDQCZH the recommendations are:


HYBRID

         asin                                              title
0  B01AB9Q2RE            Say It To Me: EP/CD-single - UK Release
1  B00000I7LP                                         Slim Shady
2  B000X1HU68                                   This Is Thirteen
3  9714721180  <span class="a-size-medium a-color-secondary a...
4  B0016DCUIA                    Invasion Of The Booty Snatchers
5  B000T31G8S                                                NaN
6  B000V66LII                                                NaN
7  B000VBH2R2                                                NaN
8  B000VIP5R4                                                NaN
9  B000ZDONPU                                                NaN
CONTENT-BASED

         asin                                              title
0  B01AB9Q2RE            Say It To Me: EP/CD-single - UK Release
1  B00000I7LP                                         Slim Shady
2  B000X1HU68   

In [23]:
compare_recommendations_for_user("AYPCUQS6ARWFH")

For user AYPCUQS6ARWFH the recommendations are:


HYBRID

         asin                                              title
0  B000BD5OPM                              Your Hit Parade: 1953
1  B01AB9Q2RE            Say It To Me: EP/CD-single - UK Release
2  B00000I7LP                                         Slim Shady
3  B000X1HU68                                   This Is Thirteen
4  9714721180  <span class="a-size-medium a-color-secondary a...
5  B0016DCUIA                    Invasion Of The Booty Snatchers
6  B000SX8ASS                                                NaN
7  B000SX9G90                                                NaN
8  B000TDWRAE                                                NaN
9  B000V64P90                                                NaN
CONTENT-BASED

         asin                                              title
0  B01AB9Q2RE            Say It To Me: EP/CD-single - UK Release
1  B00000I7LP                                         Slim Shady
2  B000X1HU68    

In [24]:
compare_recommendations_for_user("A17B6IPLJ964N0")

For user A17B6IPLJ964N0 the recommendations are:


HYBRID

         asin                                              title
0  9714721180  <span class="a-size-medium a-color-secondary a...
1  B01AB9Q2RE            Say It To Me: EP/CD-single - UK Release
2  B00000I7LP                                         Slim Shady
3  B000X1HU68                                   This Is Thirteen
4  B0016DCUIA                    Invasion Of The Booty Snatchers
5  B000S55UPC                                                NaN
6  B000TDUQ48                                                NaN
7  B000V66HBO                                                NaN
8  B000VRPFJI                                                NaN
CONTENT-BASED

         asin                                              title
0  B01AB9Q2RE            Say It To Me: EP/CD-single - UK Release
1  B00000I7LP                                         Slim Shady
2  B000X1HU68                                   This Is Thirteen
3  9714721180  <

In [25]:
compare_recommendations_for_user("AWJ9J0JAHN6PQ")

For user AWJ9J0JAHN6PQ the recommendations are:


HYBRID

         asin                                              title
0  B01AB9Q2RE            Say It To Me: EP/CD-single - UK Release
1  B00000I7LP                                         Slim Shady
2  B000X1HU68                                   This Is Thirteen
3  9714721180  <span class="a-size-medium a-color-secondary a...
4  B0016DCUIA                    Invasion Of The Booty Snatchers
5  B000QMHRMG                                                NaN
6  B000S43QJA                                                NaN
7  B000S52L5O                                                NaN
8  B000S50V9C                                                NaN
9  B000S55EQC                                                NaN
CONTENT-BASED

         asin                                              title
0  B01AB9Q2RE            Say It To Me: EP/CD-single - UK Release
1  B00000I7LP                                         Slim Shady
2  B000X1HU68    

In [26]:
compare_recommendations_for_user("AXOO7BPM22BDO")

For user AXOO7BPM22BDO the recommendations are:


HYBRID

         asin                                              title
0  B01AB9Q2RE            Say It To Me: EP/CD-single - UK Release
1  B00000I7LP                                         Slim Shady
2  B000X1HU68                                   This Is Thirteen
3  9714721180  <span class="a-size-medium a-color-secondary a...
4  B0016DCUIA                    Invasion Of The Booty Snatchers
5  B00137XCVS                                                NaN
6  B000VWOOUY                                                NaN
7  B0011Z8O2O                                                NaN
8  B0015AEF5A                                                NaN
9  B00136NI6S                                                NaN
CONTENT-BASED

         asin                                              title
0  B01AB9Q2RE            Say It To Me: EP/CD-single - UK Release
1  B00000I7LP                                         Slim Shady
2  B000X1HU68    

In [27]:
compare_recommendations_for_user("A15GBKY0IPZJI3")

For user A15GBKY0IPZJI3 the recommendations are:


HYBRID

         asin                                              title
0  B000EO8VMQ                                A Salute to America
1  9714721180  <span class="a-size-medium a-color-secondary a...
2  B01AB9Q2RE            Say It To Me: EP/CD-single - UK Release
3  B00000I7LP                                         Slim Shady
4  B000X1HU68                                   This Is Thirteen
5  B0016DCUIA                    Invasion Of The Booty Snatchers
6  B000QMG34E                                                NaN
7  B000QVL4NU                                                NaN
8  B000S43HH6                                                NaN
CONTENT-BASED

         asin                                              title
0  B01AB9Q2RE            Say It To Me: EP/CD-single - UK Release
1  B00000I7LP                                         Slim Shady
2  B000X1HU68                                   This Is Thirteen
3  9714721180  <

In [28]:
compare_recommendations_for_user("AWMORCDEUIBFO")

For user AWMORCDEUIBFO the recommendations are:


HYBRID

         asin                                              title
0  B00154RIIC                              Long Lonesome Highway
1  B00DFTQ6HY    Road Trips, Vol. 1, No. 4: From Egypt With Love
2  B0055UY6YG  Foreigner - Feels Like The First Time / Cold A...
3  B006V3XDEK   Road Trips Vol. 4 No. 5 Boston Music Hall 6/9/76
4  3426958910                    Slayer - Greatest Hits 2 CD Set
5  B000T2K374                                                NaN
6  B000WLICZC                                                NaN
7  B000WLJDSC                                                NaN
8  B001232RGE                                                NaN
9  B00124DTBK                                                NaN
CONTENT-BASED

         asin                                              title
0  B00154RIIC                              Long Lonesome Highway
1  B00DFTQ6HY    Road Trips, Vol. 1, No. 4: From Egypt With Love
2  B0055UY6YG  Fo

In [29]:
compare_recommendations_for_user("AWG2O9C42XW5G")

For user AWG2O9C42XW5G the recommendations are:


HYBRID

         asin                                              title
0  5559166928   He Touched Me: The Gospel Music of Elvis Presley
1  3426958910                    Slayer - Greatest Hits 2 CD Set
2  7799420340                                          Essential
3  B00000747R                                                   
4  9714721180  <span class="a-size-medium a-color-secondary a...
5  B01AB9Q2RE            Say It To Me: EP/CD-single - UK Release
6  B000X1HU68                                   This Is Thirteen
7  B0016DCUIA                    Invasion Of The Booty Snatchers
8  B00146B9LI                   Reaching For The Sky (LP RECORD)
CONTENT-BASED

         asin                                              title
0  B01AB9Q2RE            Say It To Me: EP/CD-single - UK Release
1  B000X1HU68                                   This Is Thirteen
2  9714721180  <span class="a-size-medium a-color-secondary a...
3  B0016DCUIA    

In [30]:
compare_recommendations_for_user("A15JTJXQXO22JJ")

For user A15JTJXQXO22JJ the recommendations are:


HYBRID

         asin                                              title
0  B01AB9Q2RE            Say It To Me: EP/CD-single - UK Release
1  B00000I7LP                                         Slim Shady
2  B000X1HU68                                   This Is Thirteen
3  B0016DCUIA                    Invasion Of The Booty Snatchers
4  B00146B9LI                   Reaching For The Sky (LP RECORD)
5  B00JGEU434  <span class="a-size-medium a-color-secondary a...
6  9714721180                      METAL RESISTANCE    BABYMETAL
7  B001EE4PUM                                                NaN
8  B0059CQ3QY                                                NaN
9  B01AN3DNE8                                                NaN
CONTENT-BASED

         asin                                    title
0  B01AB9Q2RE  Say It To Me: EP/CD-single - UK Release
1  B00000I7LP                               Slim Shady
2  B000X1HU68                         This Is 